# Part 1. EDA Analysis on Growth Metrics

**Load the data to Colab**

If you use local Python Notebook, you don't need to do that

In [0]:
from google.colab import files
uploaded = files.upload()

Saving OnlineRetail.csv to OnlineRetail.csv


In [0]:
##load packages
from datetime import datetime, timedelta
import pandas as pd
import io
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.graph_objects as go
from __future__ import division

**Load the data**

In [13]:
Retail = pd.read_csv('OnlineRetail.csv',encoding = 'unicode_escape')
Retail.head()
# Don't forget add 'encoding = 'unicode_escape', otherwise will not work

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## Revenue Analysis By Month

**Generate monthly revenue and transform variables to datetime**

In [21]:
#converting the type of Invoice Date Field from string to datetime.
Retail['InvoiceDate'] = pd.to_datetime(Retail['InvoiceDate'])
Retail.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceYearMonth
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,201012
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,201012
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,201012


In [22]:
#creating YearMonth field for the ease of reporting and visualization
Retail['InvoiceYearMonth'] = Retail['InvoiceDate'].map(lambda date: 100*date.year + date.month)
Retail.head(3)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceYearMonth
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,201012
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,201012
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,201012


In [23]:
#calculate Revenue for each row and create a new dataframe with YearMonth - Revenue columns
Retail['Revenue'] = Retail['UnitPrice'] * Retail['Quantity']
Retail_Revenue= Retail.groupby(['InvoiceYearMonth'])['Revenue'].sum().reset_index()
Retail_Revenue.head()

,InvoiceYearMonth,Revenue
0,201012,748957.020
1,201101,560000.260
2,201102,498062.650
3,201103,683267.080
4,201104,493207.121


In [25]:
#X and Y axis inputs for Plotly graph. We use Scatter for line graphs
plot_data = [
    go.Scatter(
        x=Retail_Revenue['InvoiceYearMonth'],
        y=Retail_Revenue['Revenue'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Montly Revenue'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
fig

In [28]:
import plotly
plotly.offline.plot(fig, filename = 'Revenue_Month.html', auto_open=True)

'Revenue_Month.html'

**Generate monthly change**

In [29]:
#using pct_change() function to see monthly percentage change
Retail_Revenue['MonthlyGrowth'] = Retail_Revenue['Revenue'].pct_change()

#showing first 5 rows
Retail_Revenue.head()


,InvoiceYearMonth,Revenue,MonthlyGrowth
0,201012,748957.020,NaN
1,201101,560000.260,-0.252293
2,201102,498062.650,-0.110603
3,201103,683267.080,0.371850
4,201104,493207.121,-0.278163


In [30]:
#visualization - line graph
plot_data = [
    go.Scatter(
        x=Retail_Revenue.query("InvoiceYearMonth < 201112")['InvoiceYearMonth'],
        y=Retail_Revenue.query("InvoiceYearMonth < 201112")['MonthlyGrowth'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Montly Growth Rate'
    )

fig = go.Figure(data=plot_data, layout=plot_layout)
fig

In [31]:
import plotly
plotly.offline.plot(fig, filename = 'Revenue_Growth_Month.html', auto_open=True)

'Revenue_Growth_Month.html'

We need to identify what happened on April, 2011. Was it due to less active customers or our customers did less orders? Maybe they just started to buy cheaper products?

## Monthly Active Customers

UK has the most data and we will focus on it

**Create new UK dataset**

In [38]:
#creating a new dataframe with UK customers only
Retail_uk = Retail.query("Country=='United Kingdom'").reset_index(drop=True)
Retail_uk.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceYearMonth,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,201012,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,201012,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,201012,22.00


In [39]:
#creating monthly active customers dataframe by counting unique Customer IDs
Retail_monthly_active = Retail_uk.groupby('InvoiceYearMonth')['CustomerID'].nunique().reset_index()
Retail_monthly_active.head()

,InvoiceYearMonth,CustomerID
0,201012,871
1,201101,684
2,201102,714
3,201103,923
4,201104,817


In [40]:
#plotting the output
plot_data = [
    go.Bar(
        x=Retail_monthly_active['InvoiceYearMonth'],
        y=Retail_monthly_active['CustomerID'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Active Customers'
    )

fig = go.Figure(data=plot_data, layout=plot_layout)
fig

In [41]:
plotly.offline.plot(fig, filename = 'Customer_Number_Month.html', auto_open=True)

'Customer_Number_Month.html'

In April, Monthly Active Customer number dropped to 817 from 923 

## Monthly Order Count


In [42]:
Retail_monthly_sales = Retail_uk.groupby('InvoiceYearMonth')['Quantity'].sum().reset_index()

#print the dataframe
Retail_monthly_sales.head()

#plot
plot_data = [
    go.Bar(
        x=Retail_monthly_sales['InvoiceYearMonth'],
        y=Retail_monthly_sales['Quantity'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Total # of Order'
    )

fig = go.Figure(data=plot_data, layout=plot_layout)
fig

In [43]:
plotly.offline.plot(fig, filename = 'Order_Number_Month.html', auto_open=True)

'Order_Number_Month.html'

As we expected, Order Count is also declined in April (279k to 257k, -8%)

We know that Active Customer Count directly affected Order Count decrease. At the end, we should definitely check our Average Revenue per Order as well

In [44]:
# create a new dataframe for average revenue by taking the mean of it
Retail_monthly_order_avg = Retail_uk.groupby('InvoiceYearMonth')['Revenue'].mean().reset_index()

#print the dataframe
Retail_monthly_order_avg.head()

#plot the bar chart
plot_data = [
    go.Bar(
        x=Retail_monthly_order_avg['InvoiceYearMonth'],
        y=Retail_monthly_order_avg['Revenue'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Order Average'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
fig


In [45]:
plotly.offline.plot(fig, filename = 'Ave_Order_Number_Month.html', auto_open=True)

'Ave_Order_Number_Month.html'

Even the monthly order average dropped for April (16.7 to 15.8).

## New Customer Ratio


This is a good indicator of if we are losing our existing customers or unable to attract new ones. In our dataset, we can assume a new customer is whoever did his/her first purchase in the time window we defined. We will do it monthly for this example. We will be using .min() function to find our first purchase date for each customer and define new customers based on that. 

In [49]:
#create a dataframe contaning CustomerID and first purchase date
Retail_min_purchase = Retail_uk.groupby('CustomerID').InvoiceDate.min().reset_index()
Retail_min_purchase.columns = ['CustomerID','MinPurchaseDate']
Retail_min_purchase['MinPurchaseYearMonth'] = Retail_min_purchase['MinPurchaseDate'].map(lambda date: 100*date.year + date.month)

Retail_min_purchase.head()

,CustomerID,MinPurchaseDate,MinPurchaseYearMonth
0,12346.0,2011-01-18 10:01:00,201101
1,12747.0,2010-12-05 15:38:00,201012
2,12748.0,2010-12-01 12:48:00,201012
3,12749.0,2011-05-10 15:25:00,201105
4,12820.0,2011-01-17 12:34:00,201101


**merge with UK data**

In [50]:
#merge first purchase date column to our main dataframe (tx_uk)
Retail_uk = pd.merge(Retail_uk, Retail_min_purchase, on='CustomerID')
Retail_uk.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceYearMonth,Revenue,MinPurchaseDate,MinPurchaseYearMonth
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,201012,15.30,2010-12-01 08:26:00,201012
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,201012,20.34,2010-12-01 08:26:00,201012
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,201012,22.00,2010-12-01 08:26:00,201012
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,201012,20.34,2010-12-01 08:26:00,201012
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,201012,20.34,2010-12-01 08:26:00,201012


**create a column called User Type and assign Existing**

If User's First Purchase Year Month before the selected Invoice Year Month, it is a existing customer, otherwise new

In [51]:
Retail_uk['UserType'] = 'New'
Retail_uk.loc[Retail_uk['InvoiceYearMonth']>Retail_uk['MinPurchaseYearMonth'],'UserType'] = 'Existing'
Retail_uk.head(3)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceYearMonth,Revenue,MinPurchaseDate,MinPurchaseYearMonth,UserType
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,201012,15.30,2010-12-01 08:26:00,201012,New
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,201012,20.34,2010-12-01 08:26:00,201012,New
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,201012,22.00,2010-12-01 08:26:00,201012,New


**calculate the Revenue per month for each user type**

In [52]:
Retail_user_type_revenue = Retail_uk.groupby(['InvoiceYearMonth','UserType'])['Revenue'].sum().reset_index()
#filtering the dates and plot the result
Retail_user_type_revenue = Retail_user_type_revenue.query("InvoiceYearMonth != 201012 and InvoiceYearMonth != 201112")


plot_data = [
    go.Scatter(
        x=Retail_user_type_revenue.query("UserType == 'Existing'")['InvoiceYearMonth'],
        y=Retail_user_type_revenue.query("UserType == 'Existing'")['Revenue'],
        name = 'Existing'
    ),
    go.Scatter(
        x=Retail_user_type_revenue.query("UserType == 'New'")['InvoiceYearMonth'],
        y=Retail_user_type_revenue.query("UserType == 'New'")['Revenue'],
        name = 'New'
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='New vs Existing'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
fig

In [53]:
Retail_user_type_revenue.head(3)


,InvoiceYearMonth,UserType,Revenue
1,201101,Existing,195275.51
2,201101,New,156705.77
3,201102,Existing,220994.63


In [54]:
plotly.offline.plot(fig, filename = 'User_Type_Revenue_Month.html', auto_open=True)

'User_Type_Revenue_Month.html'

Existing customers are showing a positive trend and tell us that our customer base is growing but new customers have a slight negative trend. Let’s have a better view by looking at the New Customer Ratio:


In [56]:
#create a dataframe that shows new user ratio - we also need to drop NA values (first month new user ratio is 0)
Retail_user_ratio = Retail_uk.query("UserType == 'New'").groupby(['InvoiceYearMonth'])['CustomerID'].nunique()/Retail_uk.query("UserType == 'Existing'").groupby(['InvoiceYearMonth'])['CustomerID'].nunique() 
Retail_user_ratio = Retail_user_ratio.reset_index()
Retail_user_ratio = Retail_user_ratio.dropna()
Retail_user_ratio.head(3)

,InvoiceYearMonth,CustomerID
1,201101,1.124224
2,201102,0.904000
3,201103,0.792233


In [58]:
plot_data = [
    go.Bar(
        x=Retail_user_ratio.query("InvoiceYearMonth>201101 and InvoiceYearMonth<201112")['InvoiceYearMonth'],
        y=Retail_user_ratio.query("InvoiceYearMonth>201101 and InvoiceYearMonth<201112")['CustomerID'],
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='New Customer Ratio'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
fig

New Customer Ratio has declined as expected (we assumed on Feb, all customers were New) and running around 20%

## Monthly Retention Rate


Retention rate should be monitored very closely because it indicates how sticky is your service and how well your product fits the market. For making Monthly Retention Rate visualized, we need to calculate how many customers retained from previous month.

**Monthly Retention Rate = Retained Customers From Prev. Month/Active Customers Total**


In [63]:
#identify which users are active by looking at their revenue per month
Retail_user_purchase = Retail_uk.groupby(['CustomerID','InvoiceYearMonth'])['Revenue'].sum().reset_index()
print(Retail_user_purchase.head(3))
#create retention matrix with crosstab
Retail_retention = pd.crosstab(Retail_user_purchase['CustomerID'], Retail_user_purchase['InvoiceYearMonth']).reset_index()
Retail_retention.head()

   CustomerID  InvoiceYearMonth  Revenue
0     12346.0            201101     0.00
1     12747.0            201012   706.27
2     12747.0            201101   303.04


InvoiceYearMonth,CustomerID,201012,201101,201102,201103,201104,201105,201106,201107,201108,201109,201110,201111,201112
0,12346.0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,12747.0,1,1,0,1,0,1,1,0,1,0,1,1,1
2,12748.0,1,1,1,1,1,1,1,1,1,1,1,1,1
3,12749.0,0,0,0,0,0,1,0,0,1,0,0,1,1
4,12820.0,0,1,0,0,0,0,0,0,0,1,1,0,1


**create an array of dictionary which keeps Retained & Total User count for each month**

In [64]:
months = Retail_retention.columns[2:]
months

Index([201101, 201102, 201103, 201104, 201105, 201106, 201107, 201108, 201109,
       201110, 201111, 201112],
      dtype='object', name='InvoiceYearMonth')

**Create an array of dictionary which keeps Retained & Total User count for each month**

In [0]:
retention_array = []
for i in range(len(months)-1):
    retention_data = {}
    selected_month = months[i+1]
    prev_month = months[i]
    retention_data['InvoiceYearMonth'] = int(selected_month)
    retention_data['TotalUserCount'] = Retail_retention[selected_month].sum()
    retention_data['RetainedUserCount'] = Retail_retention[(Retail_retention[selected_month]>0) & (Retail_retention[prev_month]>0)][selected_month].sum()
    retention_array.append(retention_data)
 

In [68]:
#convert the array to dataframe and calculate Retention Rate
Retail_retention = pd.DataFrame(retention_array)
Retail_retention.head()

,InvoiceYearMonth,TotalUserCount,RetainedUserCount
0,201102,714,263
1,201103,923,305
2,201104,817,310
3,201105,985,369
4,201106,943,417


In [70]:
Retail_retention['RetentionRate'] = Retail_retention['RetainedUserCount']/Retail_retention['TotalUserCount']
#plot the retention rate graph
plot_data = [
    go.Scatter(
        x=Retail_retention.query("InvoiceYearMonth<201112")['InvoiceYearMonth'],
        y=Retail_retention.query("InvoiceYearMonth<201112")['RetentionRate'],
        name="organic"
    )
    
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Monthly Retention Rate'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
fig